In [4]:
pip install gradio inference-sdk requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.12.0.88
    Uninstalling opencv-python-4.12.0.88:
      Successfully uninstalled opencv-python-4.12.0.88
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.12.15
    Uninstalling aiohttp-3.12.15:
      Successfully uninstalled aiohttp-3.12.15


In [5]:
# ===========================================
# 🌾 Crop Advisory — Disease + Weather (Urdu)
# ===========================================
import os
import requests
from datetime import datetime, timedelta

import gradio as gr
from inference_sdk import InferenceHTTPClient

# ---------------------------
# 🔐 API keys (set via env)
# ---------------------------
ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY", "").strip()
WEATHER_API_KEY  = os.getenv("WEATHER_API_KEY", "").strip()

# ---------------------------
# 🔧 Clients / Endpoints
# ---------------------------
ROBOFLOW_API_URL = "https://serverless.roboflow.com"
WEATHER_URL_NOW  = "https://api.openweathermap.org/data/2.5/weather"
WEATHER_URL_3H   = "https://api.openweathermap.org/data/2.5/forecast"

rf_client = None
if ROBOFLOW_API_KEY:
    rf_client = InferenceHTTPClient(api_url=ROBOFLOW_API_URL, api_key=ROBOFLOW_API_KEY)

# ---------------------------
# 🌐 Urdu translation helper
# ---------------------------
def translate_weather(description: str) -> str:
    mapping = {
        "clear sky": "صاف آسمان",
        "few clouds": "ہلکے بادل",
        "scattered clouds": "چھٹپٹ بادل",
        "broken clouds": "ٹوٹے ہوئے بادل",
        "shower rain": "بوندا باندی",
        "rain": "بارش",
        "moderate rain": "درمیانی بارش",
        "light rain": "ہلکی بارش",
        "thunderstorm": "آندھی اور گرج چمک",
        "snow": "برفباری",
        "mist": "دھند",
        "overcast clouds": "مکمل بادل",
    }
    return mapping.get(description.lower(), description)

# ---------------------------
# 🦠 Plant disease detection
# ---------------------------
def predict_disease(image_path: str):
    if not image_path:
        return "❌ براہ کرم پتے کی تصویر اپ لوڈ کریں"
    if not rf_client:
        return "❌ Roboflow API key سیٹ نہیں ہے"

    try:
        res = rf_client.infer(image_path, model_id="plant-disease-detection-v2-2nclk/1")
        preds = res.get("predictions", [])
        if not preds:
            return "❌ کوئی بیماری معلوم نہیں ہوئی"
        top = preds[0]
        cls = top.get("class", "نامعلوم")
        conf = float(top.get("confidence", 0.0)) * 100
        return f"🦠 بیماری: {cls}\nاعتماد: {conf:.2f}%"
    except Exception as e:
        return f"❌ خرابی: {e}"

# ---------------------------
# ☁️ Current weather (Urdu)
# ---------------------------
def get_current_weather(city: str):
    city = (city or "").strip()
    if not city:
        return "❌ براہ کرم شہر کا نام درج کریں"
    if not WEATHER_API_KEY:
        return "❌ Weather API key سیٹ نہیں ہے"

    try:
        params = {"q": city, "appid": WEATHER_API_KEY, "units": "metric", "lang": "en"}
        r = requests.get(WEATHER_URL_NOW, params=params, timeout=20)
        data = r.json()

        if data.get("cod") != 200:
            return "❌ موسم کی معلومات حاصل نہیں ہو سکیں"

        desc_en   = data["weather"][0]["description"]
        desc_ur   = translate_weather(desc_en)
        temp      = data["main"]["temp"]
        feels     = data["main"]["feels_like"]
        humidity  = data["main"]["humidity"]
        wind      = data["wind"].get("speed", 0)

        return (
            f"🌤️ موجودہ موسم: {desc_ur}\n"
            f"🌡️ درجہ حرارت: {temp}°C (محسوس: {feels}°C)\n"
            f"💧 نمی: {humidity}%\n"
            f"💨 ہوا کی رفتار: {wind} m/s"
        )
    except Exception as e:
        return f"❌ موسم کی معلومات میں خرابی: {e}"

# ---------------------------
# 📅 Tomorrow’s forecast (Urdu)
# ---------------------------
def get_tomorrow_forecast(city: str):
    city = (city or "").strip()
    if not city:
        return "❌ براہ کرم شہر کا نام درج کریں"
    if not WEATHER_API_KEY:
        return "❌ Weather API key سیٹ نہیں ہے"

    try:
        params = {"q": city, "appid": WEATHER_API_KEY, "units": "metric", "lang": "en"}
        r = requests.get(WEATHER_URL_3H, params=params, timeout=20)
        data = r.json()

        if data.get("cod") != "200":
            return "❌ شہر کا نام درست نہیں یا موسم کی معلومات دستیاب نہیں"

        tomorrow = (datetime.utcnow() + timedelta(days=1)).date()
        slots = []
        for entry in data.get("list", []):
            ts = datetime.utcfromtimestamp(entry["dt"])
            if ts.date() == tomorrow:
                desc_en = entry["weather"][0]["description"]
                slots.append({
                    "time": ts.strftime("%I:%M %p"),
                    "desc": translate_weather(desc_en),
                    "temp": entry["main"]["temp"],
                    "feels": entry["main"]["feels_like"],
                    "humidity": entry["main"]["humidity"]
                })

        if not slots:
            return "❌ کل کے لیے کوئی پیشگوئی دستیاب نہیں"

        out = [f"📅 کل ({tomorrow}) کے موسم کی پیشگوئی:\n"]
        for s in slots:
            out.append(
                f"🕒 {s['time']} — {s['desc']}\n"
                f"   🌡️ {s['temp']}°C (محسوس: {s['feels']}°C) | 💧 نمی: {s['humidity']}%"
            )
        return "\n".join(out)
    except Exception as e:
        return f"❌ خرابی: {e}"

# ---------------------------
# 🔗 Combined function
# ---------------------------
def advisory_app(image, city):
    disease = predict_disease(image)
    now     = get_current_weather(city)
    tomo    = get_tomorrow_forecast(city)
    return disease, now, tomo

# ---------------------------
# 🖥️ Gradio UI
# ---------------------------
demo = gr.Interface(
    fn=advisory_app,
    inputs=[
        gr.Image(type="filepath", label="📷 پتے کی تصویر اپ لوڈ کریں"),
        gr.Textbox(label="🏙️ شہر (مثال: Lahore, Karachi)")
    ],
    outputs=[
        gr.Textbox(label="🦠 بیماری کی پیشگوئی"),
        gr.Textbox(label="☁️ موجودہ موسم"),
        gr.Textbox(label="📅 کل کا موسم")
    ],
    title="🌾 Crop Advisory",
    description="پتوں کی بیماری کی شناخت، موجودہ موسم اور کل کی پیشگوئی — ایک ہی جگہ۔",
)

if __name__ == "__main__":
    demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8bd6918ac6e2e8af45.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
